In [2]:
import pandas as pd

In [3]:
full_data_original = pd.DataFrame(columns=["Title", "Description", "Article Text", "Keywords", "Class"])

In [4]:
for i in range(4):
    tmp = pd.read_excel("/home/alexander/HSE/Vk_hack/data.mail.xlsx", sheet_name=i)
    full_data_original = pd.concat([full_data_original, tmp])

In [5]:
#full_data_original = full_data_original.drop("URL", axis=1)

In [6]:
data_generated = pd.DataFrame(columns=["Title", "text", "description", "Keywords", "class"])

classes = ["0_3_classes", "Здравоохранение", "Образование", "Наука и технологии", "Окружающая среда и климат", "Энергетика", "Транспорт", "Сельское хозяйство", "Промышленность", "Недвижимость", "Спорт", "Развлечения", "Искусство и культура", "Мода и стиль жизни", "Туризм и путешествия", "Еда и напитки", "Автомобили", "Социальные вопросы", "Религия и верования", "Демография и население"]#, "Интернет и социальные сети", "Космос и астрономия", "Астрология и эзотерика", "История", "Знаменитости и светская жизнь", "Домашние питомцы и животные"]
for class_ in classes:
    file_name = str(class_) + ".csv"
    one_class_df = pd.read_csv(file_name)
    data_generated = pd.concat([data_generated, one_class_df])


In [7]:
data_generated = data_generated.drop('Unnamed: 0', axis=1)

In [8]:
data_generated = data_generated.rename(columns={'text': 'Article Text', 'description' : 'Description', 'class': 'Class'})

In [9]:
full_data = pd.concat([data_generated, full_data_original])

In [10]:
rambler = pd.read_csv("rambler.csv", sep=';', names = ['Title', 'Description', 'Article Text', 'Keywords'])

In [11]:
import numpy as np

rambler['Class'] = np.nan

In [12]:
full_data = pd.concat([full_data, rambler])

In [13]:
(full_data['Article Text'].isna()).sum()

0

In [14]:
kaggle_data = pd.read_csv("kaggle_full.csv")

In [15]:
kaggle_data = kaggle_data.drop(["source", "publication_date", "rubric", "subrubric"], axis=1)

In [16]:
kaggle_data = kaggle_data.rename(columns={'text': 'Article Text', 'title' : 'Title', 'class': 'Class', 'tags':'Keywords'})

In [17]:
kaggle_data['Description'] = np.nan

In [18]:
full_data = pd.concat([full_data, kaggle_data])

In [19]:
full_data.to_csv('full_data.csv')

In [20]:
full_data

,Title,Article Text,Description,Keywords,Class,URL
0,NaN,"""Новый закон о выборах вызывает оживленные ди...",NaN,NaN,Политика,NaN
1,NaN,"""Новый закон о выборах вступает в силу: избир...",NaN,NaN,Политика,NaN
2,NaN,Свежее событие в мире политики: новый закон б...,NaN,NaN,Политика,NaN
3,NaN,"""Новый закон: вводится налог на богатство""",NaN,NaN,Политика,NaN
4,NaN,В России прошло первое заседание нового прави...,NaN,NaN,Политика,NaN
...,...,...,...,...,...,...
21668,\n Россия прекратил...,\n\n \n \n \n \n \n...,NaN,NaN,NaN,NaN
21669,\n Во Владивостоке ...,\n\n \n \n \n \n \n...,NaN,NaN,NaN,NaN
21670,\n Дым от австралий...,\n\n \n \n \n \n \n...,NaN,NaN,NaN,NaN
21671,\n Около 200 жителе...,\n\n \n \n \n \n \n...,NaN,NaN,NaN,NaN


In [21]:
X, y = full_data['Article Text'], full_data['Class']

In [22]:
assert(len(np.unique(y[y.isna() == False].values)) == 29) #кайфуем

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

In [24]:
y[y.isna()] = "None"

In [25]:
y

0        Политика
1        Политика
2        Политика
3        Политика
4        Политика
           ...   
21668        None
21669        None
21670        None
21671        None
21672        None
Name: Class, Length: 80942, dtype: object

In [26]:
#target processing
y_processed = OrdinalEncoder().fit_transform(np.array(y).reshape(-1, 1)).reshape(-1)

In [27]:
print(y_processed.shape[0])
print((y_processed == 0).sum())

80942
78815


In [28]:
y_processed -= 1 #to achieve correct format for SelfTrainingClassifier

In [29]:
X

0         "Новый закон о выборах вызывает оживленные ди...
1         "Новый закон о выборах вступает в силу: избир...
2         Свежее событие в мире политики: новый закон б...
3              "Новый закон: вводится налог на богатство" 
4         В России прошло первое заседание нового прави...
                               ...                        
21668    \n\n    \n    \n        \n        \n        \n...
21669    \n\n    \n    \n        \n        \n        \n...
21670    \n\n    \n    \n        \n        \n        \n...
21671    \n\n    \n    \n        \n        \n        \n...
21672    \n\n    \n    \n        \n        \n        \n...
Name: Article Text, Length: 80942, dtype: object

In [30]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
from pymorphy2 import MorphAnalyzer
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token.isalpha() and token not in stop_words:
            token = token.strip()
            token = morph.normal_forms(token)[0]       
            tokens.append(token)
    return tokens

def pre_process(text):
    text=text.lower()
    text=re.sub("&amp;lt;/?.*?&amp;gt;"," &amp;lt;&amp;gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    # text = [word for word in text if len(word) >= 3]
    text = lemmatize(text)
    return ' '.join(text)

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


[nltk_data] Downloading package punkt to /home/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/alexander/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexander/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
#X_processed = X.apply(lambda x:pre_process(x))

In [32]:
#X_processed.to_csv("X_processed.csv")

In [33]:
X_processed = pd.read_csv("X_processed.csv")

In [34]:
full = X_processed.copy()
full['Target'] = y_processed

In [35]:
full = full.drop("Unnamed: 0", axis=1)

In [36]:
full.to_csv("full_data_with_targets.csv")

In [37]:
full_kaggle_more_labels = pd.read_csv("/home/alexander/HSE/Vk_hack/kaggle_full_more_labels.csv")
full_kaggle_more_labels = full_kaggle_more_labels[['text', 'answer']]
full_kaggle_more_labels['text'] = full_kaggle_more_labels['text'].apply(lambda x:pre_process(x))
full_kaggle_more_labels

,text,answer
0,москва апр риа новость агентство стратегически...,Экономика
1,москва фев риа новость самолёт уфа пхукет авиа...,Транспорт
2,москва фев риа новость мать петрозаводский шко...,Образование
3,москва май риа новость открытый горение молочн...,Сельское хозяйство
4,летний американский инженер любитель майкл хью...,Наука и технологии
...,...,...
4995,министерство внутренний дело белоруссия присое...,Преступность и правосудие
4996,москва мара риа новость власть колумбия депорт...,Здравоохранение
4997,москва фев риа новость затрата потребитель эле...,Энергетика
4998,волгоград авг риа новость восемь человек включ...,Транспорт


In [38]:
y = pd.concat([y, full_kaggle_more_labels['answer']])

In [39]:
X_processed = pd.concat([X_processed['Article Text'], full_kaggle_more_labels['text'] ])

In [41]:
X_processed

0       новый закон выборы вызывать оживлённый дискусс...
1       новый закон выборы вступать сила избирательный...
2       свежий событие мир политика новый закон принят...
3                   новый закон вводиться налог богатство
4       россия пройти первый заседание новый правитель...
                              ...                        
4995    министерство внутренний дело белоруссия присое...
4996    москва мара риа новость власть колумбия депорт...
4997    москва фев риа новость затрата потребитель эле...
4998    волгоград авг риа новость восемь человек включ...
4999    сотрудник авиакомпания унизить пассажирка само...
Length: 85942, dtype: object

In [42]:
data_generated_last = pd.DataFrame(columns=["Title", "text", "description", "Keywords", "class"])

classes_last = ["Интернет и социальные сети", "Космос и астрономия", "Астрология и эзотерика", "История", "Знаменитости и светская жизнь", "Домашние питомцы и животные"]
for class_ in classes:
    file_name = str(class_) + ".csv"
    one_class_df = pd.read_csv(file_name)
    data_generated_last = pd.concat([data_generated_last, one_class_df])

data_generated_last = data_generated_last.drop('Unnamed: 0', axis=1)
data_generated_last = data_generated_last.rename(columns={'text': 'Article Text', 'description' : 'Description', 'class': 'Class'})

In [43]:
data_generated_last_y = data_generated_last['Class']

In [44]:
data_generated_last_X = data_generated_last['Article Text']

In [45]:
data_generated_last_X = data_generated_last_X.apply(lambda x:pre_process(x))

In [46]:
y = pd.concat([y, data_generated_last_y])

In [47]:
X_processed = pd.concat([X_processed, data_generated_last_X])

In [48]:
final_encoder = OrdinalEncoder()
y_processed = final_encoder.fit_transform(np.array(y).reshape(-1, 1)).reshape(-1)

In [49]:
y_processed -= 1

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
vectorizer = TfidfVectorizer(min_df=0.005)
X_vectorized = vectorizer.fit_transform(X_processed)

In [52]:
X_vectorized.shape

(87069, 4750)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_processed, test_size=0.25, random_state=42)

In [54]:
y_processed

array([17., 17., 17., ...,  3.,  3.,  3.])

In [55]:
(y_train != -1).sum() / len(y_train)

0.0953890445781841

In [56]:
(y_test != -1).sum() / len(y_test)

0.09302646085997796

In [57]:
X_train.shape

(65301, 4750)

In [58]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.linear_model import LogisticRegression

self_training_model = SelfTrainingClassifier(LogisticRegression(C=10))
self_training_model.fit(X_train, y_train)

/home/alexander/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/alexander/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

SelfTrainingClassifier(base_estimator=LogisticRegression(C=10))

In [59]:
#метрика на трейне
(self_training_model.predict(X_train[y_train != -1]).ravel() == y_train[y_train != -1].ravel()).mean()

0.9486273880237598

In [60]:
#метрика на тесте
(self_training_model.predict(X_test[y_test != -1]).ravel() == y_test[y_test != -1].ravel()).mean()

0.8054320987654321

In [61]:
from sklearn.metrics import f1_score

print(f1_score(y_test[y_test != -1].ravel(), self_training_model.predict(X_test[y_test != -1]).ravel(), average='macro'))

0.6902977607042831


In [64]:
print(f1_score(y_test[y_test != -1].ravel(), self_training_model.predict(X_test[y_test != -1]).ravel(), average='micro'))

0.8054320987654321


In [75]:
((y_processed == -1) * self_training_model.predict(X_vectorized))

851886.0

In [77]:
y_full = ((y_processed == -1) * self_training_model.predict(X_vectorized)) + (y_processed != -1) * y_processed

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_full, test_size=0.25, random_state=42)

In [169]:
final_logreg = LogisticRegression(C=50, class_weight="balanced").fit(X_train, y_train)

/home/alexander/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [170]:
pred = final_logreg.predict(X_test)

In [172]:
print(f1_score(y_test.ravel(), pred, average='macro'))

0.7923853150923159


In [173]:
print(f1_score(y_test.ravel(), pred, average='micro')) #same as accuracy

0.9358232267548695


In [198]:
(y_test == 0).sum()

35

In [206]:
#input = ['Церковь храмы бог']
#input = ['Космическая ракета']
#input = ['Экологическая катастрофа произошла в Бурятии нефть вылилалсь в озеро']
#input = ['Первенцы в российской гамме — бензоэлектрические последовательные подзаряжаемые кроссоверы: средних размеров Seres M5 (6 590 000 руб.) и крупный Seres М7 (7 590 000 руб.). Да, цены велики. А все-таки не мерседесовские. Озадачило другое: «МБ Рус» несколько месяцев как продает Aito M5 и Aito M7, суть ровно те же модели с другими названиями. Позвольте, а в чем смысл запускать Seres?']
#input = ['В Башкирии в январе—марте 2024 года сельхозорганизациями было произведено 149,4 тыс. тонн молока, это на 4,1% больше, чем за аналогичный период прошлого года, следует из данных Росстата, которые изучил РБК Уфа. Республика оказалась на четвертом месте в Приволжском федеральном округе по объемам производства молока в первом квартале.В целом в ПФО за три месяца было произведено 1,6 млн тонн молока, показатели выросли на 7,1%. Лидером по росту стала Пензенская область (+30,4%), в Саратовской области и Удмуртии показатели увеличились на 14,2%.Минимальный рост зафиксирован в Оренбургской области, где производство молока в первом квартале выросло на 1,1%.По объемам производства молока в округе лидируют Татарстан (380 тыс. тонн), Удмуртия (240 тыс.) и Нижегородская область (205 тыс. тонн). Меньше всего молока произвели в первом квартале в Ульяновской (25,1 тыс.), Саратовской (36,3 тыс.) и Оренбургской (37,9 тыс.) областях.Как сообщал РБК Уфа, сельхозорганизации региона в 2023 году произвели 603,9 тыс. тонн молока. Это на 6,8% больше, чем в 2022 году, когда в республике было произведено 565,3 тыс. тонн молока. Башкирия по объему производства товарного молока за прошлый год заняла четвертое место в Приволжском федеральном округе.']

In [207]:
input_prepocessed = [pre_process(input[0])]
input_vectorized = vectorizer.transform(input_prepocessed)

In [208]:
final_logreg.predict(input_vectorized)

array([16.])

In [205]:
import joblib

In [209]:
with open('vectorizer_1.joblib.pkl', 'wb') as f:
    joblib.dump(vectorizer, f, compress=9)

In [210]:
with open('classifier_1.joblib.pkl', 'wb') as f:
    joblib.dump(final_logreg, f, compress=9)

In [178]:
class_id_to_class_mapping = {}

In [179]:
for i in range(int(y_processed.max())):
    #y[y_processed == i]
    class_id_to_class_mapping[i] = y[y_processed == i].iloc[0]

In [180]:
class_id_to_class_mapping

{0: 'Автомобили',
 1: 'Астрология и эзотерика',
 2: 'Бизнес и карьера',
 3: 'Демография и население',
 4: 'Домашние питомцы и животные',
 5: 'Еда и напитки',
 6: 'Здравоохранение',
 7: 'Знаменитости и светская жизнь',
 8: 'Интернет и социальные сети',
 9: 'Искусство и культура',
 10: 'История',
 11: 'Космос и астрономия',
 12: 'Мода и стиль жизни',
 13: 'Наука и технологии',
 14: 'Недвижимость',
 15: 'Образование',
 16: 'Окружающая среда и климат',
 17: 'Политика',
 18: 'Преступность и правосудие',
 19: 'Происшествия',
 20: 'Промышленность',
 21: 'Развлечения',
 22: 'Религия и верования',
 23: 'Сельское хозяйство',
 24: 'Социальные вопросы',
 25: 'Спорт',
 26: 'Транспорт',
 27: 'Туризм и путешествия',
 28: 'Экономика'}